# Network analysis

The NetworkInspector provides tools for quickly analyzing the created network.

First, let us create an example network:

In [1]:
import pop2net as p2n

env = p2n.Environment()
creator = p2n.Creator(env=env)

for _ in range(20):
    actor = p2n.Actor()
    actor.status = "pupil"
    env.add_actor(actor)

for _ in range(4):
    actor = p2n.Actor()
    actor.status = "teacher"
    env.add_actor(actor)


class School(p2n.LocationDesigner):
    n_locations = 2
    overcrowding = False

    def weight(self, actor):
        return 0.1

    def stick_together(self, actor):
        return actor.Classroom


class PupilsInClassroom(p2n.MeltLocationDesigner):
    n_actors = 5

    def filter(self, actor):
        return actor.status == "pupil"


class TeacherInClassroom(p2n.MeltLocationDesigner):
    n_actors = 1

    def filter(self, actor):
        return actor.status == "teacher"


class Classroom(p2n.LocationDesigner):
    def melt(self):
        return PupilsInClassroom, TeacherInClassroom


creator.create_locations(
    location_designers=[
        Classroom,
        School,
    ]
)

In the following, we initialize an instance of the NetworkInspector and bind it to our model:

In [2]:
inspector = p2n.NetworkInspector(env=env)

## Network visualization

The method `plot_bipartite_network()` plots the biparte network.
Actors are displayed as circles and locations are displayed as squares.
Using the arguments `actor_color` and `location_color`, actor and location attributes can be displayed as colors in the plot.
With the arguments `actor_attrs` and `location_attrs` you can add attributes that should be displayed in the tooltips for each node when you move the mouse pointer over it.

In the following example I display the actor attribute `status` as color and add `id_p2n` to the list of shown attribute values per actor node.

In [3]:
inspector.plot_bipartite_network(actor_color="status", actor_attrs=["id_p2n"])

You may have noticed that the weight between nodes is automatically reflected in the alpha (transparency) of the edge connecting them.

Using `plot_actor_network()`, the actor-level network is shown:

In [4]:
inspector.plot_actor_network(actor_color="status", actor_attrs=["id_p2n"])

Using `plot_networks()`, both network types are plotted:

In [5]:
inspector.plot_networks(actor_color="status", actor_attrs=["id_p2n"])

## Evaluate affiliations

Using `eval_affiliations()`, the number of locations per label, the number of actors per location and the number of locations per actor can be displayed:

In [6]:
inspector.eval_affiliations()



______________________________________
Number of locations
______________________________________

                count
location_label       
Classroom           4
School              2


______________________________________
Number of actors per location
______________________________________

                mean  std   min   25%   50%   75%   max
location_label                                         
Classroom        6.0  0.0   6.0   6.0   6.0   6.0   6.0
School          12.0  0.0  12.0  12.0  12.0  12.0  12.0


______________________________________
Number of affiliated locations per actor
______________________________________

      n_affiliated_locations
mean                     2.0
std                      0.0
min                      2.0
25%                      2.0
50%                      2.0
75%                      2.0
max                      2.0


## 

## Network measures

Using `inspector.network_measures()`, common network measures are calculated for the actor-level network.
If there are unconnected components within the network, the network measures are calculated for each component.

In [7]:
inspector.network_measures()

[{'n_nodes': 12,
  'diameter': 0.2,
  'density': 1.0,
  'transitivity': 1.0,
  'avg_clustering': 0.3472381885547425,
  'avg_path_length': 0.14545454545454548},
 {'n_nodes': 12,
  'diameter': 0.2,
  'density': 1.0,
  'transitivity': 1.0,
  'avg_clustering': 0.3472381885547425,
  'avg_path_length': 0.14545454545454548}]

## Location information

The method `inspector.location_information()` provides detailed information about the actors assigned to each location instance.  
You can specify which location types to include using the `location_labels` argument, and which actor attributes to display using the `actor_attributes` argument.  
The output can be shown as a formatted table (default) or returned as a pandas DataFrame by setting `output_format="df"`.

Below, we display the status and id of each actor in every Classroom and School location:

In [14]:
inspector.location_information(location_labels=["Classroom", "School"], actor_attributes=["status", "id_p2n"])

1.Location: Classroom 

╒════╤══════════╤══════════╤═════════════════╕
│    │ status   │   id_p2n │ location_type   │
╞════╪══════════╪══════════╪═════════════════╡
│  0 │ pupil    │        0 │ Classroom       │
├────┼──────────┼──────────┼─────────────────┤
│  1 │ pupil    │        1 │ Classroom       │
├────┼──────────┼──────────┼─────────────────┤
│  2 │ pupil    │        2 │ Classroom       │
├────┼──────────┼──────────┼─────────────────┤
│  3 │ pupil    │        3 │ Classroom       │
├────┼──────────┼──────────┼─────────────────┤
│  4 │ pupil    │        4 │ Classroom       │
├────┼──────────┼──────────┼─────────────────┤
│  5 │ teacher  │       20 │ Classroom       │
╘════╧══════════╧══════════╧═════════════════╛


2.Location: Classroom 

╒════╤══════════╤══════════╤═════════════════╕
│    │ status   │   id_p2n │ location_type   │
╞════╪══════════╪══════════╪═════════════════╡
│  0 │ pupil    │        5 │ Classroom       │
├────┼──────────┼──────────┼─────────────────┤
│  1 │ pup

You can also retrieve the information as a DataFrame for further analysis:

In [15]:
loc_inf_df = inspector.location_information(location_labels=["Classroom", "School"], actor_attributes=["status", "id_p2n"], output_format="df")
loc_inf_df.head()

,location_id,status,id_p2n,location_type
0,0,pupil,0,Classroom
1,0,pupil,1,Classroom
2,0,pupil,2,Classroom
3,0,pupil,3,Classroom
4,0,pupil,4,Classroom


## Coming soon ...

The inspector offers several additional methods for analyzing the created network.
Detailed documentation will be added soon.